In [13]:
# Instalación de paquetes necesarios (descomentar de ser necesario)

# !pip install numpy -q
# !pip install pandas -q
# !pip install matplotlib -q
# !pip install seaborn -q
# !pip install scikit-learn -q
# !pip install joblib -q

# imports

import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D # for 3D plots

import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler

In [14]:
# Cargar el dataset
df = pd.read_csv("mortalidadBogotaConPm2-5.csv")
# Visualizar las primeras filas
print("Primeras filas del dataset:")
display(df.head())

Primeras filas del dataset:


,ANIO,LOCALIDAD,N_MUERTES,NACIDOS_VIVOS,TASA POR 1000,pm2.5
0,2020,Antonio Nariño,8,995,8.0,8.7
1,2020,Barrios Unidos,13,1276,10.2,13.1
2,2020,Bosa,65,8508,7.6,0.0
3,2020,Chapinero,12,1422,8.4,6.3
4,2020,Ciudad Bolívar,88,8731,10.1,0.0


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

# --- 1. Definir umbrales para clasificaciones ---
pm_threshold = df['pm2.5'].replace(0, np.nan).median()    # ignora ceros
mort_threshold = df['TASA POR 1000'].median()

# --- 2. Clasificar exposición y mortalidad ---
df['Exposición_PM25'] = np.where(df['pm2.5'] >= pm_threshold, 'Alta', 'Baja')
df['Mortalidad'] = np.where(df['TASA POR 1000'] >= mort_threshold, 'Alta', 'Baja')

# --- 3. Tabla de contingencia 2x2 ---
tabla_contingencia = pd.crosstab(df['Exposición_PM25'], df['Mortalidad'], margins=True)
tabla_contingencia = tabla_contingencia.reset_index()
tabla_contingencia.columns.name = None

# --- 4. Relación PM2.5 vs mortalidad (para scatterplot) ---
df_pm25_mort = df[['pm2.5', 'TASA POR 1000']].dropna().copy()
df_pm25_mort.columns = ['PM2.5', 'Mortalidad']

# --- 5. Clasificación de exposición (por localidad y año) ---
df_exposicion = (
    df.groupby(['LOCALIDAD', 'ANIO'], as_index=False)
      .agg({'pm2.5': 'mean'})
)

# Categorizamos PM2.5 en niveles (puedes ajustar los umbrales)
def categoria_pm(x):
    if x >= 20: return 'Alta'
    elif x >= 10: return 'Media'
    else: return 'Baja'

df_exposicion['Categoría_PM2.5'] = df_exposicion['pm2.5'].apply(categoria_pm)
df_exposicion['Exposición_general'] = df_exposicion['Categoría_PM2.5']

# --- 6. Mapa de riesgo: casos y tasas por localidad y año ---
mapa_riesgo = (
    df.groupby(['ANIO', 'LOCALIDAD'], as_index=False)
      .agg({'N_MUERTES': 'sum', 'TASA POR 1000': 'mean'})
      .rename(columns={'N_MUERTES': 'Casos', 'TASA POR 1000': 'Tasa'})
)

# --- 7. Prueba de hipótesis (Chi-cuadrado) ---
tabla_chi2 = pd.crosstab(df['Exposición_PM25'], df['Mortalidad'])
chi2, p, dof, expected = chi2_contingency(tabla_chi2)

hipotesis = pd.DataFrame([{
    'Variable': 'PM2.5',
    'Tipo_de_prueba': 'Chi-cuadrado',
    'Estadístico': chi2,
    'p_value': p,
    'alpha': 0.05,
    'decisión': 'Rechaza H0' if p < 0.05 else 'No rechaza H0',
    'interpretación': 'Existe asociación entre PM2.5 y mortalidad' if p < 0.05 else 'No hay asociación significativa'
}])

# --- 8. Medidas de asociación (Odds Ratio y Riesgo Relativo) ---
# Extraemos los valores a, b, c, d del cruce
a = tabla_chi2.loc['Alta', 'Alta']
b = tabla_chi2.loc['Alta', 'Baja']
c = tabla_chi2.loc['Baja', 'Alta']
d = tabla_chi2.loc['Baja', 'Baja']

odds_ratio = (a * d) / (b * c)
risk_ratio = (a / (a + b)) / (c / (c + d))

medidas_asociacion = pd.DataFrame([{
    'Variable': 'PM2.5',
    'Odds_ratio': odds_ratio,
    'Risk_ratio': risk_ratio,
    'p_value': p,
    'Significancia': 'Sí' if p < 0.05 else 'No'
}])

# --- 9. Exportar para Power BI ---
os.makedirs('./estadisticos', exist_ok=True)

tabla_contingencia.to_csv('./estadisticos/tabla_contingencia.csv', index=False)
df_pm25_mort.to_csv('./estadisticos/pm25_vs_mortalidad.csv', index=False)
df_exposicion.to_csv('./estadisticos/clasificacion_exposicion.csv', index=False)
mapa_riesgo.to_csv('./estadisticos/mapa_riesgo.csv', index=False)
hipotesis.to_csv('./estadisticos/pruebas_hipotesis.csv', index=False)
medidas_asociacion.to_csv('./estadisticos/medidas_asociacion.csv', index=False)

print("✅ Archivos exportados para Power BI:")
print("tabla_contingencia.csv")
print("pm25_vs_mortalidad.csv")
print("clasificacion_exposicion.csv")
print("mapa_riesgo.csv")
print("pruebas_hipotesis.csv")
print("medidas_asociacion.csv")


✅ Archivos exportados para Power BI:
tabla_contingencia.csv, pm25_vs_mortalidad.csv, clasificacion_exposicion.csv, mapa_riesgo.csv, pruebas_hipotesis.csv, medidas_asociacion.csv
